In [7]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import datetime
import os

# Suppress TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages
tf.get_logger().setLevel('ERROR')  # Suppresses INFO and WARNING messages from TensorFlow logging

# Load the model
model = tf.keras.models.load_model('simple_model.h5',compile=False)  # Ensure correct file extension

# Define label dictionary to map class indices to person names
label_dict = {
    0: 'Muthukumar',
    1: 'Muthuram',
    2: 'Parthasarthy'
}

# Preprocess the frame to match the input size for the model
def preprocess_image(img, target_size=(224, 224)):
    img = cv2.resize(img, target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    return img_array

# Predict the person in the frame
def predict_person(img):
    img_array = preprocess_image(img)
    prediction = model.predict(img_array, verbose=0)  # Suppress progress output
    predicted_class = np.argmax(prediction, axis=1)[0]
    return predicted_class

# Mark attendance by recording detected person in a file or logging
def mark_attendance(person_label):
    try:
        # Print to console and log attendance marking
        print(f'{person_label} marked as present.')
        with open('attendance_log.txt', 'a') as f:
            f.write(f'{person_label} - {datetime.datetime.now()}\n')
    except Exception as e:
        print(f"Error logging attendance: {e}")

# Open the webcam
cap = cv2.VideoCapture(0)  # 0 is the default webcam

# Set to store detected people and avoid duplicate attendance marking
detected_people = set()

print("Press 'q' to exit the program.")

# Start a continuous loop to capture frames
while True:
    ret, frame = cap.read()  # Read a frame from the webcam
    if not ret:
        print("Failed to capture image. Exiting...")
        break

    # Predict the person in the frame
    person_id = predict_person(frame)

    # Get the person label from the dictionary
    person_label = label_dict.get(person_id, 'Unknown')

    # Mark attendance only if the person hasn't been marked yet
    if person_label not in detected_people:
        mark_attendance(person_label)
        detected_people.add(person_label)

    # Display the live feed with detected person label
    cv2.putText(frame, person_label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Show the frame with t`he label overlay
    cv2.imshow('Live Feed - Press "q" to exit', frame)

    # Check if 'q' is pressed to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Exiting the program.")
        break

# Release the capture and close any open windows
cap.release()
cv2.destroyAllWindows()


Press 'q' to exit the program.
Muthuram marked as present.
Parthasarthy marked as present.
Exiting the program.
